# Chapter 6. Algorithms

**Deutsch Algorithm**

In [10]:
def is_balance_classic(f: callable) -> bool:
  o0 = f(0)
  o1 = f(1)
  if o0 == o1:
    return False
  return True

print("lambda x: 0 ➡️", is_balance_classic(lambda x: 0))
print("lambda x: x ➡️", is_balance_classic(lambda x: x))
print("lambda x: (x + 1) % 2 ➡️", is_balance_classic(lambda x: (x + 1) % 2))
print("lambda x: 1**x ➡️", is_balance_classic(lambda x: 1**x))

lambda x: 0 ➡️ False
lambda x: x ➡️ True
lambda x: (x + 1) % 2 ➡️ True
lambda x: 1**x ➡️ False


In [11]:
def build_matrix_from_function(f: callable):
  A = []
  for i in range(2):
    A.append([])
    for j in range(2):
      if f(j) == i:
        A[i].append(1)
      else:
        A[i].append(0)
  return A

print(build_matrix_from_function(lambda x: 0))
print(build_matrix_from_function(lambda x: x))
print(build_matrix_from_function(lambda x: (x + 1) % 2))
print(build_matrix_from_function(lambda x: 1**x))

[[1, 1], [0, 0]]
[[1, 0], [0, 1]]
[[0, 1], [1, 0]]
[[0, 0], [1, 1]]


In [12]:
def build_unitary_matrix_from_function(f: callable):
  A = []
  for i1 in range(2):
    for i2 in range(2):
      A.append([])
      for j1 in range(2):
        for j2 in range(2):
          if j1 == i1 and (f(j1) + j2) % 2 == i2:
            A[-1].append(1)
          else:
            A[-1].append(0)
  return A

print(build_unitary_matrix_from_function(lambda x: 0))
print(build_unitary_matrix_from_function(lambda x: x))
print(build_unitary_matrix_from_function(lambda x: (x + 1) % 2))
print(build_unitary_matrix_from_function(lambda x: 1**x))

[[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]
[[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]]
[[0, 1, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]
[[0, 1, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]]


In [13]:
# Utility functions for matrix operations

def tensor_product(A: list[list[complex]], B: list[list[complex]]):
  res = []
  for i in range(len(A)):
    for j in range(len(B)):
      res.append([])
      for k in range(len(A[0])):
        for l in range(len(B[0])):
          res[i*len(B) + j].append(A[i][k]*B[j][l])
  return res

def multiply_matrixes(A: list[list[complex]], B: list[list[complex]]):
  ma = len(A)
  na = len(A[0])
  mb = len(B)
  nb = len(B[0])
  if na != mb:
    raise Exception("Inappropriate size!")
  res = []
  for i in range(ma):
    res.append([])
    for j in range(nb):
      sum = 0
      for k in range(na):
        sum += A[i][k]*B[k][j]
      res[i].append(sum)
  return res

def transpose_matrix(A: list[list[complex]]):
  AT = []
  for i in range(len(A[0])):
    AT.append([])
    for j in range(len(A)):
      AT[i].append(A[j][i])
  return AT

def apply_matrix(matrix, vector):
  return transpose_matrix(multiply_matrixes(matrix, transpose_matrix([vector])))[0]

In [14]:
def is_balance_quantum(F: list[list[complex]]):
  state0 = [0, 1, 0, 0] # |01>

  HH = tensor_product(
    [
      [0.5**0.5, 0.5**0.5],                                                                  
      [0.5**0.5, -0.5**0.5]
    ],
    [
      [0.5**0.5, 0.5**0.5],                                                                  
      [0.5**0.5, -0.5**0.5]
    ]
  )

  state1 = apply_matrix(HH, state0) # (|00> - |01> + |10> - |11>)/2
  
  state2 = apply_matrix(F, state1) # (-1^f(0)|0> + (-1)^f(1)|1>)(|0> - |1>)/2

  HI = tensor_product(
    [
      [0.5**0.5, 0.5**0.5],                                                                  
      [0.5**0.5, -0.5**0.5]
    ],
    [
      [1, 0],
      [0, 1]
    ]
  )

  state3 = apply_matrix(HI, state2) # (+-1)|0>(|0> - |1>)/√2 if F is constant or (+-1)|1>(|0> - |1>)/√2 if F is balanced

  print('\033[90m', f"The probabilities of the system in states |00>, |01>, |10>, |11> are {state3[0]**2}, {state3[1]**2}, {state3[2]**2}, {state3[3]**2}", '\033[0m', sep='')
  
  the_probabity_that_the_top_qubit_is_in_state_one = state3[2]**2 + state3[3]**2
  return abs(the_probabity_that_the_top_qubit_is_in_state_one - 1) < 1e-10 # the top qubit is in state |1>, conclusion: the function is balanced

print("lambda x: 0 ➡️", is_balance_quantum(build_unitary_matrix_from_function(lambda x: 0)))
print("lambda x: x ➡️", is_balance_quantum(build_unitary_matrix_from_function(lambda x: x)))
print("lambda x: (x + 1) % 2 ➡️", is_balance_quantum(build_unitary_matrix_from_function(lambda x: (x + 1) % 2)))
print("lambda x: 1**x ➡️", is_balance_quantum(build_unitary_matrix_from_function(lambda x: 1**x)))

The probabilities of the system in states |00>, |01>, |10>, |11> are 0.5000000000000002, 0.5000000000000002, 0.0, 0.0
lambda x: 0 ➡️ False
The probabilities of the system in states |00>, |01>, |10>, |11> are 0.0, 0.0, 0.5000000000000002, 0.5000000000000002
lambda x: x ➡️ True
The probabilities of the system in states |00>, |01>, |10>, |11> are 0.0, 0.0, 0.5000000000000002, 0.5000000000000002
lambda x: (x + 1) % 2 ➡️ True
The probabilities of the system in states |00>, |01>, |10>, |11> are 0.5000000000000002, 0.5000000000000002, 0.0, 0.0
lambda x: 1**x ➡️ False


**Deutsch–Jozsa Algorithm**

In [15]:
def is_balance_classic(f: callable, n: int) -> bool:
  f0 = f(0, n)
  for i in range(2**n // 2):
    if f(i + 1, n) != f0:
      return True # At least one element of the domain returns a different value => So it cannot be a const function
  return False # More than half (half + 1) elements of the domain return the same value => the remaining elements will also return the same value

from time import time

def benchmark(f: callable):
  start_time = time()
  f()
  exec_time = time() - start_time
  print('\033[90m', "Execution time: ", exec_time, "s", '\033[0m', sep='')

benchmark(lambda: print("A function which always return 0: ", is_balance_classic(lambda x, n: 0, 12)))
benchmark(lambda: print("A function which always return 1: ", is_balance_classic(lambda x, n: 1, 12)))
benchmark(lambda: print("A function returns 0 when input is even and 1 if input is odd: ", is_balance_classic(lambda x, n: x % 2, 12)))
benchmark(lambda: print("A function returns 1 for the first half of the domain and returns 0 for the other half: ", is_balance_classic(lambda x, n: 0 if x >= 2**(n - 1) else 1, 12)))

A function which always return 0:  False
Execution time: 0.00038909912109375s
A function which always return 1:  False
Execution time: 0.00015807151794433594s
A function returns 0 when input is even and 1 if input is odd:  True
Execution time: 4.76837158203125e-06s
A function returns 1 for the first half of the domain and returns 0 for the other half:  True
Execution time: 0.0006301403045654297s


In [16]:
def build_uf(f: callable, n: int):
  A = []
  for output_target in range(2**n):
    for output_control in range(2):
      A.append([])
      for input_target in range(2**n):
        for input_control in range(2):
          if output_target == input_target and (f(input_target, n) + input_control) % 2 == output_control:
            A[-1].append(1)
          else:
            A[-1].append(0)
  return A

print(build_uf(lambda x, n: 0 if x >= 2**(n - 1) else 1, 2))

[[0, 1, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 1]]


In [17]:
def check_sign(x: int, y: int):
    # Convert number to binary string
    return (-1)**(sum([int(x) for x in bin(x & y)[2:]]) % 2)

def build_hadamard(n: int):
    A = []
    for i in range(2**n):
        A.append([])
        for j in range(2**n):
            A[-1].append(2**(-n/2) * check_sign(i, j))
    return A

print(build_hadamard(2))

def build_unitary(n: int):
    A = []
    for i in range(2**n):
        A.append([])
        for j in range(2**n):
            if i == j:
                A[-1].append(1)
            else:
                A[-1].append(0)
    return A

[[0.5, 0.5, 0.5, 0.5], [0.5, -0.5, 0.5, -0.5], [0.5, 0.5, -0.5, -0.5], [0.5, -0.5, -0.5, 0.5]]


In [18]:
def deutsch_jozsa(f: callable, n: int):
    # This step would be time costly, but it is not a part of the algorithm.
    Uf = build_uf(f, n)

    # Hadamard transformation to create a superposition of all basis states
    Hn = build_hadamard(n)
    H = build_hadamard(1)

    # Start algorithm
    state = []
    for top in range(2**n):
        for bottom in range(2):
            if top == 0 and bottom == 1:
                state.append(1)
            else:
                state.append(0)
    print("|φ0> =", state)

    # Remember: states are not "cloneable", they "evolve" instead.
    state = apply_matrix(tensor_product(Hn, H), state)
    print("|φ1> =", state)

    state = apply_matrix(Uf, state)
    print("|φ2> =", state)

    state = apply_matrix(tensor_product(Hn, build_unitary(1)), state)
    print("|φ3> =", state)

    # Measure the top qubit
    probability_of_top_0 = state[0]**2 + state[1]**2 # there are 2 particular states with the top qubit in 00: |00,0> and |00,1>
    return "Constant" if probability_of_top_0 > 0 else "Balanced" # ...we will only get |0⟩ if the function is constant. If anything else is found after being measured, then the function is balanced.

print("A function which always return 0: ", deutsch_jozsa(lambda x, n: 0, 2))
print("A function which always return 1: ", deutsch_jozsa(lambda x, n: 1, 2))
print("A function returns 0 when input is even and 1 if input is odd: ", deutsch_jozsa(lambda x, n: x % 2, 2))
print("A function returns 1 for the first half of the domain and returns 0 for the other half: ", deutsch_jozsa(lambda x, n: 0 if x >= 2**(n - 1) else 1, 2))

|φ0> = [0, 1, 0, 0, 0, 0, 0, 0]
|φ1> = [0.3535533905932738, -0.3535533905932738, 0.3535533905932738, -0.3535533905932738, 0.3535533905932738, -0.3535533905932738, 0.3535533905932738, -0.3535533905932738]
|φ2> = [0.3535533905932738, -0.3535533905932738, 0.3535533905932738, -0.3535533905932738, 0.3535533905932738, -0.3535533905932738, 0.3535533905932738, -0.3535533905932738]
|φ3> = [0.7071067811865476, -0.7071067811865476, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
A function which always return 0:  Constant
|φ0> = [0, 1, 0, 0, 0, 0, 0, 0]
|φ1> = [0.3535533905932738, -0.3535533905932738, 0.3535533905932738, -0.3535533905932738, 0.3535533905932738, -0.3535533905932738, 0.3535533905932738, -0.3535533905932738]
|φ2> = [-0.3535533905932738, 0.3535533905932738, -0.3535533905932738, 0.3535533905932738, -0.3535533905932738, 0.3535533905932738, -0.3535533905932738, 0.3535533905932738]
|φ3> = [-0.7071067811865476, 0.7071067811865476, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
A function which always return 1:  Constant
|φ